# 1. 부동산 자체에 대한 정보

In [1]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt

# 문자열 길이 제한을 없앰
pd.set_option('display.max_colwidth', None)

In [2]:
# CSV 파일 목록 불러오기
folder_path = './price'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 컬럼 구조별로 대응되는 파일 목록을 저장할 딕셔너리
column_groups = {}

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    
    try:
        with open(file_path, 'r', encoding='euc-kr') as f:
            lines = f.readlines()

        header_found = False
        columns = []
        data_start_index = None

        # 파일의 각 행을 처리하여 헤더 찾기
        for i, line in enumerate(lines):
            row_data = line.strip().split(',')
            if len(row_data) > 10 and not header_found:
                columns = row_data  # 컬럼 추출
                header_found = True
                data_start_index = i + 1
                break

        # 헤더가 발견된 이후 처리
        if header_found:
            if len(lines) - 1 != i:
                cleaned_columns = [col.replace('"', '') for col in columns]
                col_tuple = tuple(cleaned_columns)
                
                # 같은 컬럼 구조를 가진 파일끼리 그룹화
                if col_tuple not in column_groups:
                    column_groups[col_tuple] = []
                column_groups[col_tuple].append((file, data_start_index))

    except Exception as e:
        print(f"Error processing file {file}: {e}")
        continue

# 결과 출력 (같은 컬럼을 가진 파일들끼리 그룹화)
for col_set, files in column_groups.items():
    print(f"Columns: {col_set}")
    for file_info in files:
        print(f"Corresponding CSV file: {file_info[0]}, Data Start Index: {file_info[1]}")
    print("\n")

Columns: ('시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)', '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지')
Corresponding CSV file: 200509.csv, Data Start Index: 16
Corresponding CSV file: 200510.csv, Data Start Index: 16
Corresponding CSV file: 200511.csv, Data Start Index: 16
Corresponding CSV file: 200609.csv, Data Start Index: 16
Corresponding CSV file: 200610.csv, Data Start Index: 16
Corresponding CSV file: 200611.csv, Data Start Index: 16
Corresponding CSV file: 200612.csv, Data Start Index: 16


Columns: ('NO', '시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)', '동', '층', '매수자', '매도자', '건축년도', '도로명', '해제사유발생일', '거래유형', '중개사소재지', '등기일자')
Corresponding CSV file: 202309.csv, Data Start Index: 16
Corresponding CSV file: 202310.csv, Data Start Index: 16
Corresponding CSV file: 202311.csv, Data Start Index: 16
Corresponding CSV file: 202312.csv, Data Start Index: 16
Corresponding CSV file: 202401.csv, Data Start Index: 16
Corresponding

In [3]:
dfs = {}

for col_set, files in column_groups.items():    
    data_list = []
    
    for file, data_start_index in files:
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_csv(file_path, encoding='euc-kr', skiprows=data_start_index, thousands=',', names=col_set)
            data_list.append(df)
        
        except Exception as e:
            continue

    if data_list:
        combined_df = pd.concat(data_list, ignore_index=True)
    else:
        combined_df = pd.DataFrame(columns=col_set)

    dfs[col_set] = combined_df

In [4]:
for col_set, df in dfs.items():
    print(col_set)
    display(df.head())

('시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)', '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지')


,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지
0,강원특별자치도 양구군 양구읍 상리,510,510,0,경림,59.400,200509,23,5000,11,1998.0,양록길23번길 11,NaN,,-,-
1,경기도 의정부시 민락동,756-7,756,7,한라비발디,84.990,200510,30,12750,19,2003.0,오목로 73,NaN,,-,-
2,경기도 의정부시 신곡동,456,456,0,신일1,59.878,200510,20,5750,3,1997.0,시민로245번길 10,NaN,,-,-
3,경기도 의정부시 신곡동,727-1,727,1,풍림,49.830,200510,20,5500,8,1998.0,회룡로 245,NaN,,-,-
4,경기도 의정부시 용현동,46,46,0,용현현대1차,129.730,200511,21,12200,11,1992.0,오목로 36,NaN,,-,-


('NO', '시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)', '동', '층', '매수자', '매도자', '건축년도', '도로명', '해제사유발생일', '거래유형', '중개사소재지', '등기일자')


,NO,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),동,층,매수자,매도자,건축년도,도로명,해제사유발생일,거래유형,중개사소재지,등기일자
0,1,대구광역시 남구 대명동,1733-3,1733,3,경일맨션,68.3000,202309,30,10000,-,3,-,-,1977,현충로 190,-,중개거래,대구 남구,23.10.25
1,2,인천광역시 중구 중산동,1887-1,1887,1,영종하늘도시한라비발디,101.5405,202309,30,36200,924,2,-,-,2012,하늘별빛로 86,-,직거래,-,23.10.30
2,3,인천광역시 미추홀구 주안동,1618,1618,0,월드스테이트(201~205),59.9960,202309,30,34900,202,19,-,-,2008,경원대로864번길 114,-,중개거래,인천 남동구 인천 미추홀구,23.11.06
3,4,서울특별시 양천구 신월동,987-1,987,1,신월시영,50.6700,202309,30,51000,8,5,-,-,1988,신월로 99,-,직거래,-,23.11.14
4,5,인천광역시 부평구 부평동,222-2,222,2,스위트홈,38.4700,202309,30,19000,스위트홈,9,-,-,2014,시장로 67,-,중개거래,인천 부평구,23.11.03


In [5]:
# 각 DataFrame을 합칠 준비
dfs_to_concat = []

# 모든 DataFrame을 리스트에 추가 (합칠 준비)
for col_set, df in dfs.items():
    dfs_to_concat.append(df)

# DataFrame을 합칠 때 column의 합집합을 유지하도록 pd.concat 사용
df = pd.concat(dfs_to_concat, axis=0, ignore_index=True, sort=False)
df.head()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,...,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,NO,동,매수자,매도자,등기일자
0,강원특별자치도 양구군 양구읍 상리,510,510,0,경림,59.400,200509,23,5000,11,...,양록길23번길 11,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
1,경기도 의정부시 민락동,756-7,756,7,한라비발디,84.990,200510,30,12750,19,...,오목로 73,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
2,경기도 의정부시 신곡동,456,456,0,신일1,59.878,200510,20,5750,3,...,시민로245번길 10,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
3,경기도 의정부시 신곡동,727-1,727,1,풍림,49.830,200510,20,5500,8,...,회룡로 245,NaN,,-,-,NaN,NaN,NaN,NaN,NaN
4,경기도 의정부시 용현동,46,46,0,용현현대1차,129.730,200511,21,12200,11,...,오목로 36,NaN,,-,-,NaN,NaN,NaN,NaN,NaN


# 2. 부동산 가격 df의 column 다듬기

In [6]:
df.columns

Index(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)',
       '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'NO', '동',
       '매수자', '매도자', '등기일자'],
      dtype='object')

In [7]:
# '시군구' 열을 빈칸 기준으로 모두 나눈 후, 첫 번째와 두 번째 값을 가져오기
split_columns = df['시군구'].str.split(' ', expand=True)

df['도/특별시/광역시'] = split_columns[0]
df['시/군/구'] = split_columns[1]
df.drop(columns=['시군구'], inplace=True)

In [8]:
df = df[['전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)', '층', '건축년도', '도/특별시/광역시', '시/군/구']]

In [10]:
# DataFrame이 슬라이스된 경우 복사
df = df.copy()

# 계약날짜 생성
df['계약날짜'] = pd.to_datetime(df['계약년월'].astype(str) + df['계약일'].astype(str), format='%Y%m%d')

# 계약년월과 계약일 열 삭제
df.drop(columns=['계약년월', '계약일'], inplace=True)

In [11]:
df.head()

,전용면적(㎡),거래금액(만원),층,건축년도,도/특별시/광역시,시/군/구,계약날짜
0,59.400,5000,11,1998.0,강원특별자치도,양구군,2005-09-23
1,84.990,12750,19,2003.0,경기도,의정부시,2005-10-30
2,59.878,5750,3,1997.0,경기도,의정부시,2005-10-20
3,49.830,5500,8,1998.0,경기도,의정부시,2005-10-20
4,129.730,12200,11,1992.0,경기도,의정부시,2005-11-21


# 3. 금리 정보 추가

In [13]:
rate_df = pd.read_csv("rate.csv")
rate_df['날짜'] = pd.to_datetime(rate_df['변경일자'].astype(str) + ' ' + rate_df['Unnamed: 1'], format='%Y %m월 %d일')
rate_df = rate_df[['날짜', '기준금리']]
rate_df.head()

,날짜,기준금리
0,2023-01-13,3.50
1,2022-11-24,3.25
2,2022-10-12,3.00
3,2022-08-25,2.50
4,2022-07-13,2.25


In [18]:
# 계약날짜에 맞는 기준금리 찾기
def get_closest_rate(contract_date):
    # contract_date 이전의 가장 가까운 날짜 찾기
    applicable_rates = rate_df[rate_df['날짜'] <= contract_date]
    
    if not applicable_rates.empty:
        return applicable_rates.iloc[0]['기준금리']  # 가장 가까운 과거의 기준금리 반환
    else:
        return None  # 만약 과거의 기준금리가 없다면 None 반환

df['기준금리'] = df['계약날짜'].apply(get_closest_rate)

# 4. 환율 정보 추가

In [45]:
# Excel 파일 읽기
exchange_df = pd.read_excel("exchange.xlsx")

# 6번째 행부터 첫 두 열을 선택
exchange_df = exchange.iloc[6:, :2]

# 인덱스 재설정
exchange_df.reset_index(drop=True, inplace=True)

# 첫 번째와 두 번째 열의 이름을 "날짜"와 "환율"로 변경
exchange_df.columns = ["날짜", "환율"]
exchange_df['날짜'] = pd.to_datetime(exchange_df['날짜'], errors='coerce')
exchange_df.head()

,날짜,환율
0,1990-03-12,698.8
1,1990-03-13,698.6
2,1990-03-14,697.9
3,1990-03-15,696.5
4,1990-03-16,696.4


In [47]:
# 계약날짜 기준으로 가장 가까운 과거의 날짜에 해당하는 환율을 찾기 위해 merge_asof 사용
df = pd.merge_asof(df.sort_values('계약날짜'), exchange_df.sort_values('날짜'),
                   left_on='계약날짜', right_on='날짜', direction='backward')
df.drop(columns=["날짜_x", "환율_x", "날짜_y", "환율_y"], inplace=True)
df.head()

,전용면적(㎡),거래금액(만원),층,건축년도,도/특별시/광역시,시/군/구,계약날짜,기준금리,날짜,환율
0,59.400,5000,11,1998.0,강원특별자치도,양구군,2005-09-23,3.25,2005-09-23,1031
1,59.878,5750,3,1997.0,경기도,의정부시,2005-10-20,3.50,2005-10-20,1055.8
2,49.830,5500,8,1998.0,경기도,의정부시,2005-10-20,3.50,2005-10-20,1055.8
3,84.990,12750,19,2003.0,경기도,의정부시,2005-10-30,3.50,2005-10-28,1042.3
4,129.730,12200,11,1992.0,경기도,의정부시,2005-11-21,3.50,2005-11-21,1039.3


# 5. 학원 정보 추가

In [53]:
a = pd.read_csv("academy.csv", encoding='cp949', low_memory=False)
a.head()

,시도교육청코드,시도교육청명,행정구역명,학원교습소구분명,학원지정번호,학원명,개설일자,등록일자,등록상태명,휴원시작일자,...,교습과정목록명,교습과정명,인당수강료내용,수강료공개여부,기숙사학원여부,시설도로명주소,시설도로명상세주소,시설도로명우편번호,전화번호,수정일자
0,B10,서울특별시교육청,강남구,학원,3000040160,대치학원,20211111,20211111,개원,,...,보습·논술,보습·논술,,Y,N,서울특별시 강남구 삼성로 347,", 201호 (대치동, 대일빌딩)",6202,02-2135-7135,20240908
1,B10,서울특별시교육청,강남구,학원,3000039285,(A STUDIO)에이스튜디오학원,20210630,20210630,개원,,...,NaN,"연기(연극,뮤지컬,오페라)",,Y,N,서울특별시 강남구 논현로134길 5,", 5층, 6층 (논현동)",6052,02-6013-8112,20231018
2,B10,서울특별시교육청,강남구,학원,2531,(주)멘토르수학전문학원,19950804,19950804,개원,,...,"보습,",보습,,N,N,서울특별시 강남구 남부순환로 2927,", 501호 일부(대치동, 대치 클래시아)",6280,02-564-5002,20240428
3,B10,서울특별시교육청,강남구,학원,3000012704,(주)산에듀김영준국어논술전문별관학원,20110415,20110415,개원,,...,보습·논술,보습·논술,,N,N,서울특별시 강남구 선릉로62길 27,", 1~4층 (대치동)",135998,02-501-0575,20240908
4,B10,서울특별시교육청,강남구,학원,3000043003,(주)산에듀김영준국어논술전문프리미엄관학원,20221230,20221230,개원,,...,보습·논술,보습·논술,,Y,N,서울특별시 강남구 선릉로62길 32-2,", 5층 (대치동, 대아빌딩)",6207,NaN,20240908
